In [1]:
%matplotlib inline

In [2]:
import os
import torch

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
from sklearn.model_selection import train_test_split

from data_generator import DataGen

In [3]:
class MMC(nn.Module):
    
    def __init__(self, beta=5, num_iter=20):
        super(MMC, self).__init__()
        self.df = 1. / beta
        self.num_iter = num_iter
        # self.weights = nn.Parameter(data= torch.tensor([[1, self.df, 0], [-1, 0, 1], [0, 0, 1]]),
        #                            requires_grad=False)
        self.weights = nn.Parameter(data=torch.from_numpy(self._init_weights()).float(), 
                                    requires_grad=False)
    
    def _init_weights(self):
        """Initializes and broadcasts weigth matrix for batch processing"""
        w = np.array([[1, self.df, 0], [-1, 0, 1], [0, 0, 1]])
        return np.broadcast_to(w, (9, 3, 3))  # 9 refers to batch size
        
    def forward(self, x):
        x = torch.bmm(torch.pow(self.weights, self.num_iter), x.unsqueeze(2)).squeeze(2)  # num_iter steps
        # x = torch.matmul(self.weights, x.transpose_(0, 1))  # single step
        return x

In [4]:
class Encoder(nn.Module):
    
    def __init__(self, num_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channel, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv4 = nn.Conv2d(8, 8, 2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(200, 128)
        self.fc2 = nn.Linear(128, 3)
        self.mmc = MMC()
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 5*5*8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # print('before: ', x)
        x = self.mmc(x)
        # print('after: ', x)
        return x

In [5]:
model = Encoder(1)
for n, p in model.named_parameters():
    print(n, p.shape)

conv1.weight torch.Size([16, 1, 3, 3])
conv1.bias torch.Size([16])
conv2.weight torch.Size([8, 16, 3, 3])
conv2.bias torch.Size([8])
conv3.weight torch.Size([8, 8, 3, 3])
conv3.bias torch.Size([8])
conv4.weight torch.Size([8, 8, 2, 2])
conv4.bias torch.Size([8])
fc1.weight torch.Size([128, 200])
fc1.bias torch.Size([128])
fc2.weight torch.Size([3, 128])
fc2.bias torch.Size([3])
mmc.weights torch.Size([9, 3, 3])


In [6]:
# summary(model, (1, 100, 100))

In [8]:
def get_model(num_channel):
    net = Encoder(num_channel=num_channel)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    return net, optimizer

In [9]:
def to_tensor(x):
    return torch.from_numpy(x).float()

def get_data(height, width, num_channel, path, size, bs):
    dgen = DataGen(height, width, num_channel)
    x, y = dgen.get_data(path, True, size, True)
    print('Data loaded...\nx:{}\ty:{}\n'.format(x.shape, y.shape))
    
    x = x / 255.
    x = x[:90]
    y = y[:90]
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)
    x_train, x_test, y_train, y_test = map(to_tensor, (x_train, x_test, y_train, y_test))
    
    train_ds = TensorDataset(x_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

    test_ds = TensorDataset(x_test, y_test)
    test_dl = DataLoader(test_ds, batch_size=bs)
    
    return train_dl, test_dl

In [10]:
def grad_flow(named_parameters):
    """To check gradient flow through the network"""
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    print('layers: {}'.format(layers))
    print('max: {}'.format(max_grads))
    print('mean: {}\n\n'.format( ave_grads))

In [11]:
def fit(net, optimizer, train_dl, test_dl, epochs):
    loss_function = nn.MSELoss()
    for epoch in range(epochs):
        net.train()
        for x, y in train_dl:
            pred = net(x)
            loss = loss_function(pred, y)

            optimizer.zero_grad()
            loss.backward()
            # grad_flow(net.named_parameters())
            optimizer.step()

        net.eval()
        with torch.no_grad():
            test_loss = sum(loss_function(net(x), y) for x, y in test_dl)
        print('epoch: {}/{} - train_loss: {:.4f} - test_loss: {:.4f}'.format(epoch + 1, epochs, loss, test_loss / len(test_dl)))

In [12]:
DATA_DIR = './data/data_simple_movement/cartesian/'
HEIGHT = 100
WIDTH = 100
NUM_CHANNEL = 2

In [13]:
net, optimizer = get_model(num_channel=2)

train_dl, test_dl = get_data(
    height=HEIGHT, 
    width=WIDTH,
    num_channel=NUM_CHANNEL,
    path=DATA_DIR,
    size=8,
    bs=9
)

fit(net, optimizer, train_dl, test_dl, epochs=100)

Data loaded...
x:(96, 2, 100, 100)	y:(96, 3)

epoch: 1/100 - train_loss: 0.3751 - test_loss: 0.3592
epoch: 2/100 - train_loss: 0.3616 - test_loss: 0.3587
epoch: 3/100 - train_loss: 0.3330 - test_loss: 0.3648
epoch: 4/100 - train_loss: 0.3379 - test_loss: 0.3661
epoch: 5/100 - train_loss: 0.1527 - test_loss: 0.3632
epoch: 6/100 - train_loss: 0.4553 - test_loss: 0.3604
epoch: 7/100 - train_loss: 0.2712 - test_loss: 0.3595
epoch: 8/100 - train_loss: 0.2686 - test_loss: 0.3608
epoch: 9/100 - train_loss: 0.1943 - test_loss: 0.3605
epoch: 10/100 - train_loss: 0.2480 - test_loss: 0.3618
epoch: 11/100 - train_loss: 0.3643 - test_loss: 0.3643
epoch: 12/100 - train_loss: 0.3403 - test_loss: 0.3622
epoch: 13/100 - train_loss: 0.2190 - test_loss: 0.3607
epoch: 14/100 - train_loss: 0.2686 - test_loss: 0.3600
epoch: 15/100 - train_loss: 0.3098 - test_loss: 0.3599
epoch: 16/100 - train_loss: 0.2635 - test_loss: 0.3604
epoch: 17/100 - train_loss: 0.4924 - test_loss: 0.3629
epoch: 18/100 - train_loss: 

In [19]:
for x, y in test_dl:
    print('prediction: {},\ntarget: {}'.format(net(x), y))

prediction: tensor([[ 0.1591,  0.3187,  0.1597],
        [ 0.2370,  0.4683,  0.2313],
        [ 0.1441,  0.2766,  0.1325],
        [-0.1475, -0.2922, -0.1447],
        [-0.0142, -0.0353, -0.0212],
        [-0.2521, -0.5059, -0.2539],
        [-0.0142, -0.0353, -0.0212],
        [ 0.2904,  0.5822,  0.2918],
        [-0.2585, -0.5175, -0.2590]], grad_fn=<SqueezeBackward1>),
target: tensor([[ 5.0227e-01,  3.6272e-03,  5.0000e-01],
        [ 8.6668e-01,  1.8136e-03,  8.6603e-01],
        [ 5.0227e-01,  3.6272e-03,  5.0000e-01],
        [-4.9545e-01,  7.2543e-03, -5.0000e-01],
        [ 7.0850e-01,  2.7204e-03,  7.0711e-01],
        [-7.0292e-01,  8.1611e-03, -7.0711e-01],
        [-9.6403e-01,  9.9747e-03, -9.6593e-01],
        [ 9.6610e-01,  9.0679e-04,  9.6593e-01],
        [-7.0292e-01,  8.1611e-03, -7.0711e-01]])
